In [103]:
import requests
import bs4
from bs4 import BeautifulSoup
import logging
logging.basicConfig(filename='webscrape.log' , format='%(asctime)s | %(levelname)s: %(message)s', level=logging.NOTSET)

In [104]:
def get_loc(result):
    try:
        return entry.find('div', {'class': 'companyLocation'}).text
    except Exception as e:
        return 'NA'


def get_comp(result):
    try:
        return result.find('span', {'class': 'companyName'}).text
    except:
        return 'NA'


def get_job(result):
    try:
        return result.find('h2', {'class': 'jobTitle'}).text
    except:
        return 'NA'


def get_sal(result):
    try:
        return result.find('div', {'class': 'salaryOnly'}).text
    except:
        return 'NA'


def get_desc(result):
    try:
        return result.find('div', {'class': 'job-snippet'}).text
    except:
        return 'NA'
def get_rating(result):
    try:
        return result.find('span', {'class': 'ratingNumber'}).text
    except:
        return 'NA'

In [105]:
from time import sleep
import pandas as pd
import numpy as np

In [106]:
job_titles = [
    'Data+Scientist', 'full+stack+developer', 'big+data+engineer',
    'software+architect', 'blockchain', 'devops+engineer', 'cloud+architect',
    'artificial+intelligence', 'product+manager'
]

len(job_titles)

9

In [121]:
results = []

In [124]:
max_results_per_job = 2000
try:
    for job in job_titles:
        print("-------------- Scrapping for : " + job  + " Started --------------------------")
        logging.info("-------------- Scrapping for : " + job  + " Started --------------------------")
        for start in range(0, max_results_per_job, 100):
            url = "https://in.indeed.com/jobs?q=" + job + "&start=" + str(start) + '&limit=100'
            html = requests.get(url) 
            soup = BeautifulSoup(html.text, 'html.parser')
            for result in soup.find_all('div', {'class':'job_seen_beacon'}):
                
                results.append(result)
            sleep(5)
        logging.info("Gathered " + str(len(results)) + " jobs so far ...")
        print("Gathered " + str(len(results)) + " jobs so far ...")
        print("-------------- Scrapping for : " + job  + " Completed --------------------------")
        logging.info("-------------- Scrapping for : " + job  + " Completed -------------------------")
    print("Scraping done !!! Check logs for more info")
except Exception as e:
    logging.error(e)
    print("Something went wrong :(. Please check logs")

-------------- Scrapping for : Data+Scientist Started --------------------------
Gathered 1276 jobs so far ...
-------------- Scrapping for : Data+Scientist Completed --------------------------
-------------- Scrapping for : full+stack+developer Started --------------------------
Gathered 1909 jobs so far ...
-------------- Scrapping for : full+stack+developer Completed --------------------------
-------------- Scrapping for : big+data+engineer Started --------------------------
Gathered 2484 jobs so far ...
-------------- Scrapping for : big+data+engineer Completed --------------------------
-------------- Scrapping for : software+architect Started --------------------------
Gathered 3104 jobs so far ...
-------------- Scrapping for : software+architect Completed --------------------------
-------------- Scrapping for : blockchain Started --------------------------
Gathered 4104 jobs so far ...
-------------- Scrapping for : blockchain Completed --------------------------
------------

In [125]:
jobs0 = pd.DataFrame(columns=['location', 'title', 'company', 'salary', 'summary', 'rating'])

In [126]:
for entry in results:
    location = get_loc(entry)
    title = get_job(entry)
    company = get_comp(entry)
    salary = get_sal(entry)
    desc = get_desc(entry)
    rating = get_rating(entry)
    jobs0.loc[len(jobs0)] = [location, title, company, salary, desc, rating]

In [175]:
jobs0.drop_duplicates(inplace = True, keep='first')
jobs0
jobs0.to_csv(r'./Dataset/job_raw.csv')
# jobs0.describe()

In [138]:
len(jobs0) - len(jobs0[jobs0.duplicated()])

5418

In [ ]:
# start = 10
# url = "https://in.indeed.com/jobs?q=Data+Scientist&start=" + str(start) + '&limit=100'

# html = requests.get(url)
# results = []  
# soup = BeautifulSoup(html.text, 'html.parser')
# # print(soup)
# for result in soup.find_all('div', {'class':'job_seen_beacon'}):
#     results.append(result)
#     print(result)
# sleep(1)

In [142]:
len(results)

7072

In [141]:
jobs0[jobs0.duplicated()]

,location,title,company,salary,summary,rating


In [253]:
display(jobs0.describe())
print(jobs0.isna().sum())

,location,title,company,salary,summary,rating,clean_location
count,5418,5418,5418,1316,5418,3026,5418
unique,385,3409,2426,604,4868,30,207
top,"Bengaluru, Karnataka",DevOps Engineer,IBM,Full-time,\nSoftware Developers at IBM are the backbone ...,3.9,"Bengaluru, Karnataka"
freq,1426,165,362,248,82,695,1737


location             0
title                0
company              0
salary            4102
summary              0
rating            2392
clean_location       0
dtype: int64


In [147]:
jobs0.replace(to_replace='NA', value=np.nan, inplace=True)

In [166]:
df = jobs0

In [169]:
for col in df:
    print(col, " ", df[col].unique())

location   ['Bengaluru, Karnataka' 'Noida, Uttar Pradesh'
 'Chennai, Tamil Nadu+1 location' 'Chennai, Tamil Nadu'
 'Bengaluru, Karnataka+1 location' 'Gurgaon, Haryana'
 'Hyderabad, Telangana+1 location'
 'Temporarily Remote in Bengaluru, Karnataka' 'Mumbai, Maharashtra'
 'Pune, Maharashtra' 'Remote' 'Remote in Bengaluru, Karnataka'
 'Kalyan Nagar, Bengaluru, Karnataka' 'India' 'Hyderabad, Telangana'
 'New Delhi, Delhi' 'Chandigarh, Chandigarh'
 'Pune, Maharashtra+1 location' 'Bellandur, Bengaluru, Karnataka'
 'Hybrid remote in Gurgaon, Haryana' 'Ahmedabad, Gujarat'
 'Remote in India' 'Hybrid remote in Noida, Uttar Pradesh'
 'Delhi, Delhi+1 location' 'Navi Mumbai, Maharashtra'
 'Sholinganallur, Chennai, Tamil Nadu+1 location'
 'Remote in Mumbai, Maharashtra+7 locations'
 'Coimbatore South, Tamil Nadu' 'West Bengal'
 'Remote in Ahmedabad, Gujarat' 'Remote in Chennai, Tamil Nadu'
 'Delhi, Delhi' 'Temporarily Remote in Chennai, Tamil Nadu'
 'Temporarily Remote in Kolkata, West Bengal' 'Rem

In [178]:
df

,location,title,company,salary,summary,rating
0,"Bengaluru, Karnataka",Data Scientist,Applied Materials Inc.,"₹5,26,846 - ₹17,85,107 a year","\nAcquire, cleanse, organize and mine massive ...",3.9
1,"Bengaluru, Karnataka",Senior Data Scientist,CORESTRAT LABS PRIVATE LIMITED,"₹30,00,000 a year",\nCollaborate to enhance data infrastructure a...,NaN
2,"Noida, Uttar Pradesh",Tech Mahindra hiring for Data Scientist - Noida,Tech Mahindra,"₹4,00,000 - ₹16,00,000 a year","\nData scientist: 3 years (Required).\nPython,...",NaN
3,"Bengaluru, Karnataka",newData Scientist,Happay,"₹8,00,000 - ₹12,00,000 a year","\n1-2 Years as a data scientist, machine learn...",NaN
4,"Chennai, Tamil Nadu+1 location",newAssociate Data Scientist,Shell,NaN,"\nGathers data, analyses and reports findings....",3.9
...,...,...,...,...,...,...
7056,"Vikhroli, Mumbai, Maharashtra",Team Lead Designer,Accelya,NaN,\nExperienced in agile ways of working and abl...,3.6
7060,"Hyderabad, Telangana",newSenior Full Stack Web Developer,Avidbots,NaN,\nDesign the technical specification to match ...,NaN
7062,"Bengaluru, Karnataka",Inside Sales Account Management,Dell Technologies,NaN,\nMay include some enterprise products and ser...,4.0
7064,"Bengaluru, Karnataka",Technical Staff Software Engineer,Dell Technologies,NaN,"\nResearch, evaluate and contribute to project...",4.0


In [192]:
print('Unique Locations  \t:', len(df.location.unique()))
print('Unique Title  \t\t:', len(df.title.unique()))
print('Unique Company  \t:', len(df.company.unique()))
print('Unique Salary \t\t:', len(df.salary.unique()))
print('Unique Rating \t\t:', len(df.rating.unique()))
# Unique Locations  	: 385
# Unique Title  		: 3409
# Unique Company  	: 2426
# Unique Salary 		: 605
# Unique Rating 		: 31

Unique Locations  	: 385
Unique Title  		: 3409
Unique Company  	: 2426
Unique Salary 		: 605
Unique Rating 		: 31


In [193]:
location  = df.location

In [198]:
"""
    * Noise found
         1. 'Remote in '
         2. '+1 location'
         3. 'Hybrid remote in '
         4. '+7 location'
         5. '+2 location'
         6. 'Temporarily Remote in '

"""

for i in location:
    print(i)

Bengaluru, Karnataka
Bengaluru, Karnataka
Noida, Uttar Pradesh
Bengaluru, Karnataka
Chennai, Tamil Nadu+1 location
Bengaluru, Karnataka
Chennai, Tamil Nadu
Chennai, Tamil Nadu
Bengaluru, Karnataka
Bengaluru, Karnataka
Bengaluru, Karnataka+1 location
Bengaluru, Karnataka
Gurgaon, Haryana
Hyderabad, Telangana+1 location
Temporarily Remote in Bengaluru, Karnataka
Mumbai, Maharashtra
Bengaluru, Karnataka
Chennai, Tamil Nadu
Pune, Maharashtra
Chennai, Tamil Nadu
Remote
Noida, Uttar Pradesh
Bengaluru, Karnataka
Remote in Bengaluru, Karnataka
Bengaluru, Karnataka
Gurgaon, Haryana
Chennai, Tamil Nadu+1 location
Bengaluru, Karnataka
Bengaluru, Karnataka+1 location
Bengaluru, Karnataka+1 location
Bengaluru, Karnataka
Bengaluru, Karnataka
Bengaluru, Karnataka
Bengaluru, Karnataka
Mumbai, Maharashtra
Bengaluru, Karnataka
Remote
Gurgaon, Haryana
Kalyan Nagar, Bengaluru, Karnataka
Gurgaon, Haryana
Bengaluru, Karnataka
Bengaluru, Karnataka
Mumbai, Maharashtra
Bengaluru, Karnataka
Bengaluru, Karnataka

Bengaluru, Karnataka
Chennai, Tamil Nadu
Chennai, Tamil Nadu
Pune, Maharashtra
Chennai, Tamil Nadu
Pune, Maharashtra
Bengaluru, Karnataka
Bengaluru, Karnataka
Kolkata, West Bengal+1 location
Bengaluru, Karnataka
Remote in Mumbai, Maharashtra+1 location
Chennai, Tamil Nadu
Jaipur, Rajasthan+1 location
Hyderabad, Telangana+1 location
Indore, Madhya Pradesh
Bengaluru, Karnataka
Ahmedabad, Gujarat
Chennai, Tamil Nadu
Bengaluru, Karnataka
Hyderabad, Telangana
Raipur, Chhattisgarh
Hyderabad, Telangana
New Delhi, Delhi
New Delhi, Delhi
Bengaluru, Karnataka+2 locations
Bengaluru, Karnataka
Bhubaneshwar, Orissa
Salem, Tamil Nadu
Remote in Pune, Maharashtra
Bengaluru, Karnataka
Chennai, Tamil Nadu
Gurgaon, Haryana
Ahmedabad, Gujarat
Bengaluru, Karnataka
Coimbatore, Tamil Nadu
Bengaluru, Karnataka
Noida, Uttar Pradesh
Delhi, Delhi
Hyderabad, Telangana
Yusaf Sarai, Delhi
Pune, Maharashtra+4 locations
Hyderabad, Telangana
Pimpri-Chinchwad, Maharashtra
Pune, Maharashtra
Remote in Jaipur, Rajasthan
M

Temporarily Remote in Navi Mumbai, Maharashtra
Hyderabad, Telangana
Bengaluru, Karnataka+1 location
Noida, Uttar Pradesh+1 location
Chandigarh, Chandigarh
Bengaluru, Karnataka
Hyderabad, Telangana+1 location
Bengaluru, Karnataka
Bengaluru, Karnataka+2 locations
Kochi, Kerala
Mumbai, Maharashtra
Bengaluru, Karnataka
New Delhi, Delhi
Chennai, Tamil Nadu
Chennai, Tamil Nadu
Remote
India
Bengaluru, Karnataka
Bengaluru, Karnataka
Bengaluru, Karnataka
Bengaluru, Karnataka+4 locations
Bengaluru, Karnataka
Ahmedabad, Gujarat
Pune, Maharashtra
Bengaluru, Karnataka+2 locations
Hyderabad, Telangana+1 location
Hyderabad, Telangana
Hyderabad, Telangana+3 locations
Bengaluru, Karnataka
Bengaluru, Karnataka
Bengaluru, Karnataka
Mumbai, Maharashtra
Chennai, Tamil Nadu
Hyderabad, Telangana
Hyderabad, Telangana
Noida, Uttar Pradesh
Orissa
Bengaluru, Karnataka+1 location
Vadodara, Gujarat
Bengaluru, Karnataka
Bengaluru, Karnataka
Remote in India
Hyderabad, Telangana
Mumbai, Maharashtra
Indore, Madhya Pra

In [199]:
import nltk.corpus
nltk.download('stopwords')
from nltk.corpus import stopwords



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kisho\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [243]:
stop_words = stopwords.words('english') +  ['in', 'remote', 'Remote', 'location', 'locations','Temporarily', 'Hybrid', 'India']
df['clean_location'] = df['location'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

In [258]:
df

,location,title,company,salary,summary,rating,clean_location
0,"Bengaluru, Karnataka",Data Scientist,Applied Materials Inc.,"₹5,26,846 - ₹17,85,107 a year","\nAcquire, cleanse, organize and mine massive ...",3.9,"Bengaluru, Karnataka"
1,"Bengaluru, Karnataka",Senior Data Scientist,CORESTRAT LABS PRIVATE LIMITED,"₹30,00,000 a year",\nCollaborate to enhance data infrastructure a...,NaN,"Bengaluru, Karnataka"
2,"Noida, Uttar Pradesh",Tech Mahindra hiring for Data Scientist - Noida,Tech Mahindra,"₹4,00,000 - ₹16,00,000 a year","\nData scientist: 3 years (Required).\nPython,...",NaN,"Noida, Uttar Pradesh"
3,"Bengaluru, Karnataka",newData Scientist,Happay,"₹8,00,000 - ₹12,00,000 a year","\n1-2 Years as a data scientist, machine learn...",NaN,"Bengaluru, Karnataka"
4,"Chennai, Tamil Nadu+1 location",newAssociate Data Scientist,Shell,NaN,"\nGathers data, analyses and reports findings....",3.9,"Chennai, Tamil Nadu"
...,...,...,...,...,...,...,...
7056,"Vikhroli, Mumbai, Maharashtra",Team Lead Designer,Accelya,NaN,\nExperienced in agile ways of working and abl...,3.6,"Vikhroli, Mumbai, Maharashtra"
7060,"Hyderabad, Telangana",newSenior Full Stack Web Developer,Avidbots,NaN,\nDesign the technical specification to match ...,NaN,"Hyderabad, Telangana"
7062,"Bengaluru, Karnataka",Inside Sales Account Management,Dell Technologies,NaN,\nMay include some enterprise products and ser...,4.0,"Bengaluru, Karnataka"
7064,"Bengaluru, Karnataka",Technical Staff Software Engineer,Dell Technologies,NaN,"\nResearch, evaluate and contribute to project...",4.0,"Bengaluru, Karnataka"


In [250]:
print('Uncleaned Locations  \t:', len(df.location.unique()))
print('Cleaned Locations  \t:', len(df.clean_location.unique()))

Uncleaned Locations  	: 385
Cleaned Locations  	: 207


In [257]:
for i in df.clean_location:
    print("'", i, "'")

' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
' Noida, Uttar Pradesh '
' Bengaluru, Karnataka '
' Chennai, Tamil Nadu '
' Bengaluru, Karnataka '
' Chennai, Tamil Nadu '
' Chennai, Tamil Nadu '
' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
' Gurgaon, Haryana '
' Hyderabad, Telangana '
' Bengaluru, Karnataka '
' Mumbai, Maharashtra '
' Bengaluru, Karnataka '
' Chennai, Tamil Nadu '
' Pune, Maharashtra '
' Chennai, Tamil Nadu '
'  '
' Noida, Uttar Pradesh '
' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
' Gurgaon, Haryana '
' Chennai, Tamil Nadu '
' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
' Mumbai, Maharashtra '
' Bengaluru, Karnataka '
'  '
' Gurgaon, Haryana '
' Kalyan Nagar, Bengaluru, Karnataka '
' Gurgaon, Haryana '
' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
' M

' Bengaluru, Karnataka '
' Mumbai, Maharashtra '
' Bengaluru, Karnataka '
' Chennai, Tamil Nadu '
' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
' Agra, Uttar Pradesh '
' Ahmedabad, Gujarat '
' Sholinganallur, Chennai, Tamil Nadu '
' Bengaluru, Karnataka '
' Hyderabad, Telangana '
' Bengaluru, Karnataka '
' Hyderabad, Telangana '
' Chandigarh, Chandigarh '
' Chennai, Tamil Nadu '
' Gurgaon, Haryana '
' Mumbai, Maharashtra '
' Chennai, Tamil Nadu '
' Bengaluru, Karnataka '
' Hyderabad, Telangana '
' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
' Hyderabad, Telangana '
' Hyderabad, Telangana '
' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
' Kathivakkam, Chennai, Tamil Nadu '
' Hyderabad, Telangana '
' Hyderabad, Telangana '
' Pune, Maharashtra '
' Kochi, Kerala '
' Bengaluru, Karnataka '
' Mumbai, Maharashtra '
' Bengaluru, Karnataka '
' Chennai, Tamil Nadu '
' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
' Agra, Uttar Pradesh '
' Ahmedabad, Gujarat '
' Sholinganallur, Chenn

' Chennai, Tamil Nadu '
' Bengaluru, Karnataka '
' Tamil Nadu '
' Bengaluru, Karnataka '
' Delhi, Delhi '
' Gurgaon, Haryana '
' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
'  '
' Bengaluru, Karnataka '
' Ahmedabad, Gujarat '
' Chennai, Tamil Nadu '
' Pune, Maharashtra '
' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
' Mumbai, Maharashtra '
' Bengaluru, Karnataka '
' Chennai, Tamil Nadu '
' Bengaluru, Karnataka '
' Mumbai, Maharashtra '
' Hyderabad, Telangana '
' Pune, Maharashtra '
' Pune, Maharashtra '
' Bengaluru, Karnataka '
' Mumbai, Maharashtra '
' Gurgaon, Haryana '
' Hyderabad, Telangana '
' Bengaluru, Karnataka '
' Hyderabad, Telangana '
' Pune, Maharashtra '
' Bengaluru, Karnataka '
' Pune, Maharashtra '
' Jaipur, Rajasthan '
' Bengaluru, Karnataka '
' Hinjewadi, Pune, Maharashtra '
' Chennai, Tamil Nadu '
' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
' Hyderabad, Telangana '
' Bengaluru, Karnataka '
' Pune, Maharashtra '
' Chennai, Tamil N

' Bengaluru, Karnataka '
' Hyderabad, Telangana '
' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
' Gurgaon, Haryana '
' Mumbai, Maharashtra '
' Hyderabad, Telangana '
' Hyderabad, Telangana '
' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
' Hyderabad, Telangana '
' Hyderabad, Telangana '
' Pune, Maharashtra '
' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
' Hyderabad, Telangana '
' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
' Chennai, Tamil Nadu '
' Bengaluru, Karnataka '
' Hyderabad, Telangana '
' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
' Hyderabad, Telangana '
' Gurgaon, Haryana '
' Chennai, Tamil Nadu '
' Hyderabad, Telangana '
' Kochi, Kerala '
' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
' Hyderabad, Telangana '
' Hyderabad, Telangana '
' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
' Pune, Maharashtra '

'  '
' Bengaluru, Karnataka '
' HSR Layout, Bengaluru, Karnataka '
'  '
' Indore, Madhya Pradesh '
' Pune, Maharashtra '
' Chennai, Tamil Nadu '
' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
'  '
'  '
' Noida, Uttar Pradesh '
' Chennai, Tamil Nadu '
'  '
' Bengaluru, Karnataka '
' Gurgaon, Haryana '
' Chennai, Tamil Nadu '
' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
'  '
' Hyderabad, Telangana '
' Delhi, Delhi '
' Delhi, Delhi '
'  '
' Kolkata, West Bengal '
' Bengaluru, Karnataka '
' Baner, Pune, Maharashtra '
' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
'  '
'  '
' Chennai, Tamil Nadu '
' Egmore, Chennai, Tamil Nadu '
' Hyderabad, Telangana '
' Hyderabad, Telangana '
' Gachibowli, Hyderabad, Telangana '
' Bengaluru, Karnataka '
' Gautam Budh Nagar, Uttar Pradesh '
' Hyderabad, Telangana '
' Chennai, Tamil Nadu '
'  '
' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
' Hyderabad, Telangana '
'  '
' Noida, Uttar Pradesh '
' Kerala '
' Bengaluru, Karnataka '
'  '
' Bengal

' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
' Hyderabad, Telangana '
' Gurgaon, Haryana '
'  '
' Gurgaon, Haryana '
' Hyderabad, Telangana '
' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
' Pune, Maharashtra '
' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
' Hyderabad, Telangana '
' Chennai, Tamil Nadu '
' Delhi '
' Bengaluru, Karnataka '
' Kolkata, West Bengal '
' Chennai, Tamil Nadu '
'  '
'  '
' Thiruvananthapuram, Kerala '
' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
' Mumbai, Maharashtra '
' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
' Delhi, Delhi '
' Kochi, Kerala '
' Pune, Maharashtra '
' Chennai, Tamil Nadu '
' Mumbai, Maharashtra '
' Mumbai, Maharashtra '
' Gurgaon, Haryana '
' Kolkata, West Bengal '
' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
' Pune, Maharashtra '
' Chennai, Tamil Nadu '
' Chennai, Tamil Nadu '
' Pune, Mahar

' Bengaluru, Karnataka '
' Kolkata, West Bengal '
' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
' Chennai, Tamil Nadu '
' Mysore, Karnataka '
' Hyderabad, Telangana '
' Kolkata, West Bengal '
' Pune, Maharashtra '
' Bengaluru, Karnataka '
' Mumbai, Maharashtra '
' Chandigarh, Chandigarh '
' Chennai, Tamil Nadu '
' Bengaluru, Karnataka '
' Pune, Maharashtra '
' Bengaluru, Karnataka '
' Hyderabad, Telangana '
' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
' Pune, Maharashtra '
' Bengaluru, Karnataka '
' Hyderabad, Telangana '
'  '
' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
' Mumbai, Maharashtra '
' Mumbai, Maharashtra '
' Bengaluru, Karnataka '
' Pune, Maharashtra '
' Gurgaon, Haryana '
' Noida, Uttar Pradesh '
' Bengaluru, Karnataka '
' Mumbai, Maharashtra '
' Bengaluru, Karnataka '
' Pune, Maharashtra '
' Chennai, Tamil Nadu '
' Pune, Maharashtra '
' Bengaluru, Karnataka '
' Bengaluru, Karnataka '


' Pune, Maharashtra '
' Bengaluru, Karnataka '
' Mumbai, Maharashtra '
' Pune, Maharashtra '
' New Delhi, Delhi '
' Noida, Uttar Pradesh '
' Hyderabad, Telangana '
'  '
' Bengaluru, Karnataka '
' Mumbai, Maharashtra '
' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
' Bengaluru, Karnataka '
' Vikhroli, Mumbai, Maharashtra '
' Hyderabad, Telangana '
' Kollam, Kerala '
' Bengaluru, Karnataka '
' Hyderabad, Telangana '
' Pune, Maharashtra '
' Pune, Maharashtra '
' Hyderabad, Telangana '
' Pune, Maharashtra '
' Noida, Uttar Pradesh '
' Bengaluru, Karnataka '
' Pune, Maharashtra '
' Chennai, Tamil Nadu '
' Kuala, Rajasthan '
' Sholinganallur, Chennai, Tamil Nadu '
' Jalgaon, Maharashtra '
' Mumbai, Maharashtra '
' Pune, Maharashtra '
' Kollam, Kerala '
' Chandigarh, Chandigarh '
' Chennai, Tamil Nadu '
' Nagpur, Maharashtra '
' Vikhroli, Mumbai, Maharashtra '
' Vikhroli, Mumbai, Maharashtra '
' Gurgaon, Haryana '
' Telangana '
' Bengaluru, Karnataka '
' Pune, Maharashtra '
' Telangana '
' 

In [249]:
def clean_loc(location):
    remove_list = ['+1', '+2', '+3','+4','+5','+6','+7','+8', '+9']
    for i in remove_list:
        if i in location:
            return location.replace(i, '')
    return location
df.clean_location = df.clean_location.apply(clean_loc)
df.clean_location = df.clean_location.str.strip()

In [255]:
df.clean_location.isnull().sum()

0